In [1]:
import pylab as plt
import numpy as np
from astropy.io import fits
import matplotlib as mpl
import math
import astropy.io.fits as fits
from astropy.table import Table 

In [2]:
def roi_cut(xcut,ycut,x,y):
    """
    xcut : numpy array
         Array of x-values for the cut
    ycut : numpy array
         Array of y-values for the cut
    x : numpy array or list
         Array of x-values that should be cut
    y : numpy array or list
         Array of y-values that should be cut

    Returns
    ind : numpy array
       The indices of values OUTSIDE the cut
    cutind : 
       The indices of values INSIDE the cut

    """

    from matplotlib.path import Path
    tupVerts = list(zip(xcut,ycut))
    points = np.vstack((x,y)).T
    p = Path(tupVerts) # make a polygon
    inside = p.contains_points(points)
    ind, = np.where(~inside)
    cutind, = np.where(inside)

    return ind, cutind

In [7]:
def LMCParallax (ra,dec,radius): 
    df=Table(fits.getdata ('/Users/mgkadrmas/research/LMCquery.fits.gz')) #query
    xcut=[0.762097,0.883065,1.312500,1.372984,1.457661,1.633065,1.826613,2.008065,1.983871,1.536290,1.403226,1.197581,1.070565,0.810484] #define x coordinates of polygon
    ycut=[19.049242,19.409091,19.465909,18.613636,17.856061,17.117424,16.473485,16.113636,15.469697,15.583333,16.303030,17.287879,17.969697,18.613636] #define y coordinates of polygon
    x=df['bp_rp'] 
    y=df['phot_g_mean_mag']
    (ind, cutind) = roi_cut(xcut,ycut,x,y) 
    df=df[cutind] #only data inside of the polygon!!
    medpmra=np.nanmedian(df['pmra'])
    medpmdec=np.nanmedian(df['pmdec'])
    pndist=np.sqrt((df['pmra']-medpmra)**2 + (df['pmdec']-medpmdec)**2)
    good= (np.abs(df['parallax'])<=0.2) & (pndist<=2.0)
    medplx=np.nanmedian(df['parallax'][good])+0.030112982075019102 #right offset?
    ngood=np.sum(good)
    Sd= (1/ngood)*math.sqrt(sum((medplx-(df['parallax'][good]))**2)) #standard deviation of mean
    sigma= (1/ngood)*np.sqrt(np.sum(df['parallax_error']**2)) #standard error of parallax
    print(ra, dec, medplx)
    return (ngood, medplx, Sd, sigma)
    

In [8]:
#expanded with a larger range: dec -80 to -55 #ra 55 to 110 

radius=0.5
dtype = np.dtype([('ra',float),('dec',float),('radius',float),('ngood',int),('parallax',float),('Sd',float),('sigma',float)])
cat = np.zeros([56,26],dtype=dtype)
for i in range (56):
    ra=i+55
    for j in range (26):
        dec=j-80
        (ngood, medplx, Sd, sigma) = LMCParallax(ra,dec,radius) 
        cat[i,j]['ra']=ra
        cat[i,j]['dec']=dec
        cat[i,j]['radius']=radius
        cat[i,j]['ngood']=ngood
        cat[i,j]['parallax']=medplx
        cat[i,j]['Sd']=Sd
        cat[i,j]['sigma']=sigma
        

/anaconda3/lib/python3.7/site-packages/astropy/table/column.py:965: RuntimeWarning: invalid value encountered in less_equal
  return getattr(self.data, op)(other)


55 -80 0.03442405468903541
55 -79 0.03442405468903541
55 -78 0.03442405468903541
55 -77 0.03442405468903541
55 -76 0.03442405468903541
55 -75 0.03442405468903541
55 -74 0.03442405468903541


KeyboardInterrupt: 